## Algorítmo de resolução do problema proposto com restrições adicionais

Biblioteca usada para resolver o problema:

    scipy - biblioteca do google

Restrições do problema:
    
    1. O melhor resultado deve conciliar os menores custos com os maiores retornos
    2. Se o investimento 1 for selecionado, então o investimento 5 não deve ser;
    3. Se o investimento 2 for selecionado, então o investimento 4 também deve ser.
    4. Dado o perfil do investidor, alguns investimentos estariam disponíveis e outros não com base no risco relacionado a cada um.

In [2]:
%pip install pulp


[notice] A new release of pip available: 22.1.2 -> 23.0
[notice] To update, run: C:\Users\maxna\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install unitest

^C
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 2
  ╰─> [84 lines of output]
           -----------------------------------      1.0/1.1 MB 13.6 kB/s eta 0:00:10
      ERROR: Exception:
      Traceback (most recent call last):
        File "C:\Users\maxna\AppData\Local\Temp\pip-standalone-pip-d8npbeyk\__env_pip__.zip\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
          yield
        File "C:\Users\maxna\AppData\Local\Temp\pip-standalone-pip-d8npbeyk\__env_pip__.zip\pip\_vendor\urllib3\response.py", line 516, in read
          data = self._fp.read(amt) if not fp_closed else b""
        File "C:\Users\maxna\AppData\Local\Temp\pip-standalone-pip-d8npbeyk\__env_pip__.zip\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
          data = self.__fp.read(amt)
        File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\http\client.p

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 720.6/720.6 kB 827.0 kB/s eta 0:00:00
     -------------------------------------- 56.9/56.9 kB 992.3 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 49.8 kB/s eta 0:00:00
     ---------------------------------------- 62.8/62.8 kB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 71.2 kB/s eta 0:00:00
     -------------------------------------- 40.9/40.9 kB 494.4 kB/s eta 0:00:00
     -------------------------------------- 79.5/79.5 kB 745.1 kB/s eta 0:00:00
     ------------------------------------ 101.9/101.9 kB 975.1 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'error'

[notice] A new release of pip available: 22.1.2 -> 23.0
[notice] To update, run: C:\Users\maxna\AppData\Lo

# Declaração da classse de dados

In [3]:
# Classe de dados

from dataclasses import dataclass

@dataclass
class ValuableItem:
    opcao: str
    value: float
    retorno_esperado: float
    risco: int

    @property
    def value_razao(self) -> float:
        "Returns retorno esperado / value"
        return self.retorno_esperado / (self.value + 1e-9)

### Função de montagem da tabela

In [4]:
import pandas as pd 
from typing import List

def items_to_table(opcao: List[ValuableItem]) -> pd.DataFrame:
  records = [{
          'Opção': i.opcao,
          'Custo ($)': i.value,
          'Retorno esperado ($)': i.retorno_esperado,
          'Risco de investimento (%)': i.risco
  } for i in opcao]
  records.append({
    'Opcao': 'Total',
    'Valor ($)': sum(i.value for i in opcao),
    'Retorno esperado ($)': sum(i.retorno_esperado for i in opcao),
    'Risco de investimento (%)': sum(i.risco for i in opcao)
  })
  return pd.DataFrame.from_records(records)

### Declaração dos dados de entrada

In [5]:
investimento = 1000000

custo_acao = [470006,400000,176000,270000,340000,230000,50000,440000] #pesos

retorno_esperado = [410000,330000,140000,250000,326000,326000,90000,190006] # utilidade

risco = [90, 85, 20, 45, 30, 26, 10, 80] # risco de investimento

available_items = [ValuableItem(f'opcao {i+1}', c, w, r) for i, (c, w, r) in enumerate(zip(custo_acao, retorno_esperado, risco))] 

items_to_table(available_items)

,Opção,Custo ($),Retorno esperado ($),Risco de investimento (%),Opcao,Valor ($)
0,opcao 1,470006.0,410000,90,NaN,NaN
1,opcao 2,400000.0,330000,85,NaN,NaN
2,opcao 3,176000.0,140000,20,NaN,NaN
3,opcao 4,270000.0,250000,45,NaN,NaN
4,opcao 5,340000.0,326000,30,NaN,NaN
5,opcao 6,230000.0,326000,26,NaN,NaN
6,opcao 7,50000.0,90000,10,NaN,NaN
7,opcao 8,440000.0,190006,80,NaN,NaN
8,NaN,NaN,2062006,386,Total,2376006.0


In [8]:
# Declarando as variáveis de forma conjunta
from pulp import *

# Variáveis usadas durante o programa para armazenar os investimentos escolhidos
opcao = []
custoResult = []
retornoResult = []
riscoResult = []

# Função que executa a otimização
def optimizer (perfil):
        
    # Define as variáveis
    x = [] # Variáveis de decisão
    funcao_objetivo = 0 # Variável que armazena a função objetivo
    restricaoCusto = 0 # Variável que armazena a restrição de custo
    restricaoRisco = [] # Vetor de variáveis que armazena as restrições de risco
    restricaoCond1 = 0
    restricaoCond2 = 0


    ptsMax = 0 # Controla o risco máximo com base no perfil de investidor
    
    tamanho = len(retorno_esperado) # Variável de controle do número de investimentos disponíveis

    # Determinando as variáveis de decisão
    for i in range(tamanho):
        x.append(LpVariable(("x"+str(i)),0,1,LpInteger))

    # Cria um problema de otimização em maximização
    prob = LpProblem("Otimização carteira de investimentos", LpMaximize)
    prob.solver = GLPK_CMD() # Determina o solver GLPK (instalado na máquina)

    # Montagem da função objetivo
    for i in range(tamanho):
        funcao_objetivo+=(retorno_esperado[i]*x[i])
        
    # Montagem da restrição de custo
    for i in range(tamanho):
        restricaoCusto+=(custo_acao[i]*x[i])
        
    # Decidindo qual o perfil de investidor
    if perfil == "a": # Conservador
        ptsMax = 40
    elif perfil == "b": # Moderado
        ptsMax = 70
    elif perfil == "c": # Arrojado
        ptsMax = 99
    else:
        print("O perfil selecionado não é válido!") # Caso o perfil selecionado não seja válido
        return 1
    
    # Adicionando as restrições de risco    
    for i in range(tamanho):
        restricaoRisco.append((risco[i]*x[i])) 
        
    # Adicionando as restrições condicionais
    if (tamanho >= 5): # Caso existam 5 ou mais investimentos
        restricaoCond1 = (x[0] + x[4])
        restricaoCond2 = (x[1] + x[3])
            
    # Define a função objetivo
    prob+=funcao_objetivo

    # Define as restrições
    prob+=restricaoCusto<=investimento # Restrição de custo
    for i in range(tamanho):
        prob+=restricaoRisco[i]<=ptsMax # Restrições de risco para cada investimento
    
    if (tamanho >= 5): # Caso existam 5 ou mais investimentos
        prob+=restricaoCond1<=1
        prob+=restricaoCond1<=1

    # Resolve o problema
    prob.solve()
    
    # Caso tudo dê certo, a função irá estruturar e mostrar os valores ao usuário
    if (prob.status == 1):
        imprimeResultado(perfil,x,prob)
        return prob.solve() # Retorno da função para definição de sucesso do plano
    else:
        print("Não foi possível obter um resultado, confira os dados de entrada!!!")

# Função de montagem e impressão dos resultados
def imprimeResultado (perfil,x,prob):
    
    # Imprime o resultado
    print("Perfil de investimento do cliente: ", end="")
    if perfil == "a":
        print("Conservador\n")
    elif perfil == "b":
        print("Moderado\n")
    elif perfil == "c":
        print("Arrojado\n")
        
    print("Investimentos escolhidos:")
    
    tamanho = len(x) # Variável de controle do número de investimentos disponíveis

    # Mostra quais foram os investimentos selecionados
    for i in range(tamanho):
        if x[i].varValue == 1:
            print("Opção", str(i+1), end = ", ")
            
    print("\n") # Quebra de linha
    
    # Preenche os vetores com seus respectivos dados para os investimentos escolhidos
    for i in range(tamanho):
        if x[i].varValue == 1:
            opcao.append(i)
            custoResult.append(custo_acao[i])
            retornoResult.append(retorno_esperado[i])
            riscoResult.append(risco[i])
    
    # Mostra o retorno esperado dos investimentos
    print("Retorno Esperado: ", value(prob.objective))


# Início do programa

# Verificação do perfil do investidor
perfil = str(input('Insira o perfil de investimento do cliente: \n  a -> Conservador(risco < 40%)\n  b -> Moderado(risco < 70%)\n c -> Arrojado(risco <= 99%)\nTipo de perfil:'))

#Chamada da função para determinar quais os investimentos indicados
optimizer(perfil)   

# Com os valores dos investimentos em mãos é possível montar uma tabela que mostra quais os investimentos selecionados e suas características
resultado = [ValuableItem(f'opcao {opcao[i]+1}', c, w, r) for i, (c, w, r) in enumerate(zip(custoResult, retornoResult, riscoResult))]

# Monta e mostra a tabela de resultados
items_to_table(resultado)

Perfil de investimento do cliente: Moderado

Investimentos escolhidos:
Opção 4, Opção 5, Opção 6, Opção 7, 

Retorno Esperado:  992000


C:\Users\maxna\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


,Opção,Custo ($),Retorno esperado ($),Risco de investimento (%),Opcao,Valor ($)
0,opcao 4,270000.0,250000,45,NaN,NaN
1,opcao 5,340000.0,326000,30,NaN,NaN
2,opcao 6,230000.0,326000,26,NaN,NaN
3,opcao 7,50000.0,90000,10,NaN,NaN
4,NaN,NaN,992000,111,Total,890000.0


# Testes unitários

In [13]:
import unittest

def soma(a, b):
    return a + b

class TestSoma(unittest.TestCase):
    def test_soma(self):
        result = soma(1, 2)
        self.assertEqual(result, 3)

if __name__ == '__main__':
    unittest.main()

usage: ipykernel_launcher.py [-h] [-v] [-q] [--locals] [-f] [-c] [-b]
                             [-k TESTNAMEPATTERNS]
                             [tests ...]
ipykernel_launcher.py: error: argument -f/--failfast: ignored explicit argument 'c:\\Users\\maxna\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-16268snVr3LDB4auW.json'


AssertionError: 

In [7]:
import unittest

class TestOptimizer(unittest.TestCase):
    def test_optimizer_perfil_a(self):
        perfil = "a"
        custo_acao = [1, 2, 3]
        retorno_esperado = [20, 24, 53]
        risco = [10, 50, 90]
        investimento = 5
        expected_output = 1
        self.assertEqual(optimizer(perfil), expected_output)

    # def test_optimizer_perfil_b(self):
    #     perfil = "b"
    #     retorno_esperado = [0.1, 0.2, 0.3]
    #     custo_acao = [1, 2, 3]
    #     risco = [10, 20, 30]
    #     investimento = 5
    #     expected_output = "Perfil de investimento do cliente: Moderado\n\nInvestimentos escolhidos:\nOpção 1, \n\n"
    #     self.assertEqual(optimizer(perfil), expected_output)

    # def test_optimizer_perfil_c(self):
    #     perfil = "c"
    #     retorno_esperado = [0.1, 0.2, 0.3]
    #     custo_acao = [1, 2, 3]
    #     risco = [10, 20, 30]
    #     investimento = 5
    #     expected_output = "Perfil de investimento do cliente: Arrojado\n\nInvestimentos escolhidos:\nOpção 1, \n\n"
    #     self.assertEqual(optimizer(perfil), expected_output)

    # def test_optimizer_perfil_invalido(self):
    #     perfil = "d"
    #     retorno_esperado = [0.1, 0.2, 0.3]
    #     custo_acao = [1, 2, 3]
    #     risco = [10, 20, 30]
    #     investimento = 5
    #     expected_output = "O perfil selecionado não é válido!"
    #     self.assertEqual(optimizer(perfil), expected_output)
        

if __name__ == '__name__':
    unittest.main()